<h1 style="background-color:#000000;font-family:newtimeroman;font-size:250%;text-align:center;border-radius: 15px 50px;"><span style="color:#FFFFFF">Harry Potter Sentiment Analysis</span></h1>



- [1. Libraries](#1)

- [2. Loading Data](#2)

- [3. Preprocessing](#3)
    
- [4. Exploratory Data Analysis](#4)
    * [4.1 General Statistics](#4.1)
    * [4.2 Character Appearance Frequency](#4.2)
   
- [5. Sentiment Analysis](#5)
    * [5.1 Sentence](#5.1)
    * [5.2 Chapter](#5.2)
    * [5.3 Entire Series](#5.2)

- [6. LDA Topic Modeling](#6)
- [7. ](#7)

<a id="1"></a>

<h1 style="background-color:#740001;font-family:newtimeroman;font-size:225%;text-align:center;border-radius:15px 50px;"><span style="color:#EEBA30">Libraries</span></h1>

In [ ]:
import warnings
warnings.filterwarnings(action = "ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import json
import seaborn as sns
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import corpus
from nltk.corpus import stopwords
import spacy
nlp = spacy.load('en_core_web_md')
from collections import Counter
import re
from operator import itemgetter
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.summarization import summarize

import string
from string import punctuation
import os
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob

OSError: ignored

<a id="2"></a>

<h1 style="background-color:#740001;font-family:newtimeroman;font-size:225%;text-align:center;border-radius:15px 50px;"><span style="color:#EEBA30">Loading Data</span></h1>

In [ ]:
# Load character list
characters_list=pd.read_csv("characters_list.csv")

FileNotFoundError: ignored

In [ ]:
# Create a list for all books
all_books = ['book_1.txt',
             'book_2.txt',
             'book_3.txt',
             'book_4.txt',
             'book_5.txt',
             'book_6.txt',
             'book_7.txt'
]

# Load books
def load_book(book_path):
    with open(book_path,encoding='utf8') as input_file:
        return input_file.read()

def load_all_books():
    books = []
    for book in all_books:
        books.append(load_book(book))
    return books

book_1 = load_book(all_books[0])
book_2 = load_book(all_books[1])
book_3 = load_book(all_books[2])
book_4 = load_book(all_books[3])
book_5 = load_book(all_books[4])
book_6 = load_book(all_books[5])
book_7 = load_book(all_books[6])

In [ ]:
# Prepare data
def prep(text):
    text = re.sub("<[^>]*>", " ", text) # Remove HTML tags
    text = re.sub(r"\\w\w\w\\w\w\w"," ",text) # Remove redundant characters
    text = " ".join(text.split()) # Remove extra spaces, tabs, and new lines
    return text

book_1 = prep(book_1)
book_2 = prep(book_2)
book_3 = prep(book_3)
book_4 = prep(book_4)
book_5 = prep(book_5)
book_6 = prep(book_6)
book_7 = prep(book_7)

In [ ]:
# Load chapter names
chapters_file = 'chapters.json'

def load_chapter_names():
    with open(chapters_file) as input_file:
        chapters=json.load(input_file)
    return chapters

chapter_names = load_chapter_names()

def load_chapters(book, chapter_names):
    book_chapters = []
    current_chapter = chapter_names[0]
    current_chapter_location = len(current_chapter) + 1
    for chapter in chapter_names[1:]:
        # Find where next chapter starts
        next_chapter_location = book.find(chapter)
        # Get chapter text -- all text from the location of current chapter till the start of the next chapter
        book_chapters.append(book[current_chapter_location:next_chapter_location])
        # Move start pointer to the next chapter
        current_chapter_location = next_chapter_location + len(chapter)
        # Save next chapter name
        current_chapter = chapter
    book_chapters.append(book[current_chapter_location:])
    return book_chapters

def load_all_chapters():
    book_chapters = []
    chapter_names = load_chapter_names()
    for book_number, book in enumerate(load_all_books()):
        chapters = load_chapters(book, chapter_names[book_number])
        book_chapters.append(chapters)
    return book_chapters, chapter_names

In [ ]:
book_1_chapters, book_1_chapter_names = load_chapters(book_1, chapter_names[0]), chapter_names[0]
book_2_chapters, book_2_chapter_names = load_chapters(book_2, chapter_names[1]), chapter_names[1]
book_3_chapters, book_3_chapter_names = load_chapters(book_3, chapter_names[2]), chapter_names[2]
book_4_chapters, book_4_chapter_names = load_chapters(book_4, chapter_names[3]), chapter_names[3]
book_5_chapters, book_5_chapter_names = load_chapters(book_5, chapter_names[4]), chapter_names[4]
book_6_chapters, book_6_chapter_names = load_chapters(book_6, chapter_names[5]), chapter_names[5]
book_7_chapters, book_7_chapter_names = load_chapters(book_7, chapter_names[6]), chapter_names[6]

In [ ]:
# Tokenize the sentences
def tokenizeSentence(book, sent_list):
    for i in range(len(book)):
        sentence = sent_tokenize(book[i])
        sent_list += sentence

book_1_sent = []
tokenizeSentence(book_1_chapters, book_1_sent)

book_2_sent = []
tokenizeSentence(book_2_chapters, book_2_sent)

book_3_sent = []
tokenizeSentence(book_3_chapters, book_3_sent)

book_4_sent = []
tokenizeSentence(book_4_chapters, book_4_sent)

book_5_sent = []
tokenizeSentence(book_5_chapters, book_5_sent)

book_6_sent = []
tokenizeSentence(book_6_chapters, book_6_sent)

book_7_sent = []
tokenizeSentence(book_7_chapters, book_7_sent)

In [ ]:
# Create a dataframe to store all sentences from 7 books
def createDF(sent_list, df, num):
    df = pd.DataFrame()
    df = pd.DataFrame(sent_list, columns = ["sentence"]) 
    df["book"] = num
    return df

df = pd.DataFrame()
df1 = createDF(book_1_sent,df,1)
df2 = createDF(book_2_sent,df,2)
df3 = createDF(book_3_sent,df,3)
df4 = createDF(book_4_sent,df,4)
df5 = createDF(book_5_sent,df,5)
df6 = createDF(book_6_sent,df,6)
df7 = createDF(book_7_sent,df,7)

df=pd.concat([df1,df2,df3,df4,df5,df6,df7],axis=0)
# df.reset_index(drop=True, inplace=True)

<a id="3"></a>

<h1 style="background-color:#740001;font-family:newtimeroman;font-size:225%;text-align:center;border-radius:15px 50px;"><span style="color:#EEBA30">Preprocessing</span></h1>

In [ ]:
# Stopwords for wordcloud for topic modeling

stpwrd = nltk.corpus.stopwords.words("english")
custom_stop_words = ["harry","potter","hermione","ron","j","k","rowling","s","t","said","page","professor","know",
                     "back","i","j.k","like","could",'would',"philosophers","stone","chamber","secrets","prisoner",
                     "azbakan","goblet","fire","order","phoenix","half","blood","prince","deathly","hallows",
                     "looked","one","got","get","see","going","go","told","look","looking","tell","saw","think",
                     "around","though","even","still"]
all_stpwrd = stpwrd + custom_stop_words

In [ ]:
def preprocess_text(text):
    text = text.lower()  # Lowercase text
    text = re.sub(r"[!#$+-@\']/g","", text) # remove special characters and punctuations
    text = re.sub(f"[{re.escape(punctuation)}]", "", text)  # Remove punctuation
    text = re.sub("<[^>]*>", " ", text) # Replace HTML tags with space - this needs to be before removing special characters
    text = re.sub(r"\W"," ",text) # Replace non-alphanumerics with space
    text = re.sub(r"\s+[a-z]\s+"," ", text) # Remove single character
    text = re.sub(r"\s+"," ",text)  # Replace multiple spaces with one space
#     text = re.sub(r'(.)\1{3,}',r'\1', text) # Remove repeated characters like hellooo 
#     text = re.sub(r"[^A-Za-z0-9\s]+", " ", text) # Replace special characters and punctuation with space
#     text = " ".join(text.split())  # Remove extra spaces, tabs, and new lines
    text = remove_stopwords(text)
    text = lemmatize(text)
    return text

def remove_stopwords(text):
    lst = []
    for token in text.split():
        if token not in stpwrd:
            lst.append(token)
    return " ".join(lst)

def lemmatize(text):
    doc = nlp(text)
    lemmas = " ".join([token.lemma_ for token in doc])
    return lemmas

In [ ]:
# def chapter_prep(book):
#     book_prep=[]
#     for sentence in book:
#         sentence = preprocess_text(sentence)
#         sentence=remove_stopwords(sentence)
#         sentence=lemmatize(sentence)
#         book_prep.append(sentence)
#     return book_prep

In [ ]:
# book1_chapters_prep=chapter_prep(book1chapters)
# book1=s_replace(book1_chapters_prep)
# book2_chapters_prep=chapter_prep(book2chapters)
# book2=s_replace(book2_chapters_prep)
# book3_chapters_prep=chapter_prep(book3chapters)
# book3=s_replace(book3_chapters_prep)
# book4_chapters_prep=chapter_prep(book4chapters)
# book4=s_replace(book4_chapters_prep)
# book5_chapters_prep=chapter_prep(book5chapters)
# book5=s_replace(book5_chapters_prep)
# book6_chapters_prep=chapter_prep(book6chapters)
# book6=s_replace(book6_chapters_prep)
# book7_chapters_prep=chapter_prep(book7chapters)
# book7=s_replace(book7_chapters_prep)

In [ ]:
book_1_prep = preprocess_text(book_1)
book_2_prep = preprocess_text(book_2)
book_3_prep = preprocess_text(book_3)
book_4_prep = preprocess_text(book_4)
book_5_prep = preprocess_text(book_5)
book_6_prep = preprocess_text(book_6)
book_7_prep = preprocess_text(book_7)

<a id="4"></a>

<h1 style="background-color:#740001;font-family:newtimeroman;font-size:225%;text-align:center;border-radius:15px 50px;"><span style="color:#EEBA30">EDA</span></h1>

<a id="4.1"></a>

<h1 style="background-color:#1A472A;font-family:newtimeroman;font-size:200%;text-align:center;border-radius:15px 50px;"><span style="color:#DCDCDC">General Statistics</span></h1>

In [ ]:
book_title = ["Book 1" , "Book 2", "Book 3", "Book 4" , "Book 5", "Book 6" , "Book 7"]

# book_title = ["Harry Potter and the Sorcerer's Stone",
#               "Harry Potter and the Chamber of Secrets",
#               "Harry Potter and the Prisoner of Azkaban",
#               "Harry Potter and the Goblet of Fire",
#               "Harry Potter and the Order of the Phoenix",
#               "Harry Potter and the Half-Blood Prince",
#               "Harry Potter and the Deathly Hallows"]

In [ ]:
# Get number of chapters per book
countChap = {book_title[0]: len(book_1_chapters),
             book_title[1]: len(book_2_chapters),
             book_title[2]: len(book_3_chapters),
             book_title[3]: len(book_4_chapters),
             book_title[4]: len(book_5_chapters),
             book_title[5]: len(book_6_chapters),
             book_title[6]: len(book_7_chapters)}

In [ ]:
# Get number of sentences per book
countSent = {book_title[0]: len(book_1_sent),
             book_title[1]: len(book_2_sent),
             book_title[2]: len(book_3_sent),
             book_title[3]: len(book_4_sent),
             book_title[4]: len(book_5_sent),
             book_title[5]: len(book_6_sent),
             book_title[6]: len(book_7_sent)}

In [ ]:
# Get number of words per book
words1 = word_tokenize(book_1)
words2 = word_tokenize(book_2)
words3 = word_tokenize(book_3)
words4 = word_tokenize(book_4)
words5 = word_tokenize(book_5)
words6 = word_tokenize(book_6)
words7 = word_tokenize(book_7)

countWord = {book_title[0]: len(book_1),
             book_title[1]: len(book_2),
             book_title[2]: len(book_3),
             book_title[3]: len(book_4),
             book_title[4]: len(book_5),
             book_title[5]: len(book_6),
             book_title[6]: len(book_7)}

In [ ]:
# Plot the statistics
fig, ax = plt.subplots(1,3)
sns.set(rc={'figure.figsize':(7,20)})
plt.ticklabel_format(style='plain', axis='y')
sns.barplot(x=list(countChap.keys()),y=list(countChap.values()),ax=ax[0]).set_title("Number of Chapters in Each Book")
sns.barplot(x=list(countSent.keys()),y=list(countSent.values()),ax=ax[1]).set_title("Number of Sentences in Each Book")
sns.barplot(x=list(countWord.keys()),y=list(countWord.values()),ax=ax[2]).set_title("Number of Words in Each Book")

<a id="4.1"></a>

<h1 style="background-color:#1A472A;font-family:newtimeroman;font-size:200%;text-align:center;border-radius:15px 50px;"><span style="color:#DCDCDC">N-Grams</span></h1>

In [ ]:
def compute_ngrams(sequence, n):
    return list(
            zip(*(sequence[index:] 
                     for index in range(n)))
    )

def get_top_ngrams(corpus, ngram_val, limit):
    tokens = nltk.word_tokenize(corpus)
    ngrams = compute_ngrams(tokens, ngram_val)
    ngrams_freq_dist = nltk.FreqDist(ngrams)
    sorted_ngrams_fd = sorted(ngrams_freq_dist.items(), 
                              key=itemgetter(1), reverse=True)
    sorted_ngrams = sorted_ngrams_fd[0:limit]
    sorted_ngrams = [(' '.join(text), freq) 
                     for text, freq in sorted_ngrams]

    return sorted_ngrams

In [ ]:
get_top_ngrams(corpus=book_1_prep, ngram_val=2,limit=10)
# get_top_ngrams(corpus=book_1_prep, ngram_val=3,limit=10)

In [ ]:
get_top_ngrams(corpus=book_2_prep, ngram_val=2,limit=10)
# get_top_ngrams(corpus=book_2_prep, ngram_val=3,limit=10)

In [ ]:
get_top_ngrams(corpus=book_3_prep, ngram_val=2,limit=10)
# get_top_ngrams(corpus=book_3_prep, ngram_val=3,limit=10)

In [ ]:
get_top_ngrams(corpus=book_4_prep, ngram_val=2,limit=10)
# get_top_ngrams(corpus=book_4_prep, ngram_val=3,limit=10)

In [ ]:
get_top_ngrams(corpus=book_5_prep, ngram_val=2,limit=10)
# get_top_ngrams(corpus=book_5_prep, ngram_val=3,limit=10)

In [ ]:
get_top_ngrams(corpus=book_6_prep, ngram_val=2,limit=10)
# get_top_ngrams(corpus=book_6_prep, ngram_val=3,limit=10)

In [ ]:
get_top_ngrams(corpus=book_7_prep, ngram_val=2,limit=10)
# get_top_ngrams(corpus=book_7_prep, ngram_val=3,limit=10)

<a id="4.2"></a>

<h1 style="background-color:#1A472A;font-family:newtimeroman;font-size:200%;text-align:center;border-radius:15px 50px;"><span style="color:#DCDCDC">Character Appearance Frequency</span></h1>

In [ ]:
# Get character's first names
characters_list["first_name"] = characters_list['name'].apply(lambda s: s.split()[0].lower())

# Get character's last names
characters_list["last_name"] = characters_list['name'].apply(lambda s: s.split()[-1].lower())

In [ ]:
# Convert all tokenized words to lowercase
words1 = [x.lower() for x in words1]
words2 = [x.lower() for x in words2]
words3 = [x.lower() for x in words3]
words4 = [x.lower() for x in words4]
words5 = [x.lower() for x in words5]
words6 = [x.lower() for x in words6]
words7 = [x.lower() for x in words7]

# Creat a count dictionary for each book
countDict1 = Counter(words1)
countDict2 = Counter(words2)
countDict3 = Counter(words3)
countDict4 = Counter(words4)
countDict5 = Counter(words5)
countDict6 = Counter(words6)
countDict7 = Counter(words7)

In [ ]:
def charFreq(countDict, num):
    bowDF = pd.DataFrame(countDict.items()).rename(columns={0:'word',1:'count'})
    
    fig, ax = plt.subplots(1,2)
    plt.figure(figsize=(15,8))
    
    fn = bowDF[bowDF['word'].isin(characters_list["first_name"])].sort_values(by='count',ascending=False).iloc[0:10]
    sns.barplot(x = fn['word'],y = fn['count'], ax=ax[0]).set_title(f"Character Appearance in Book {num} by First Name")

    ln = bowDF[bowDF['word'].isin(characters_list["last_name"])].sort_values(by='count',ascending=False).iloc[0:10]
    sns.barplot(x = ln['word'],y = ln['count'], ax=ax[1]).set_title(f"Character Appearance in Book {num} by Last Name")

In [ ]:
charFreq(countDict1,1)
charFreq(countDict2,2)
charFreq(countDict3,3)
charFreq(countDict4,4)
charFreq(countDict5,5)
charFreq(countDict5,5)
charFreq(countDict6,6)
charFreq(countDict7,7)

<a id="5"></a>

<h1 style="background-color:#740001;font-family:newtimeroman;font-size:225%;text-align:center;border-radius:15px 50px;"><span style="color:#EEBA30">Sentiment Analysis</span></h1>

<a id="5.1"></a>

<h1 style="background-color:#1A472A;font-family:newtimeroman;font-size:200%;text-align:center;border-radius:15px 50px;"><span style="color:#DCDCDC">Sentiment Analysis For Each Sentence</span></h1>

In [ ]:
sentiment_model=SentimentIntensityAnalyzer()

In [ ]:
def sent_by_sent(sent_list):
    for sentence in sent_list:
        scores = sentiment_model.polarity_scores(sentence)
        print(sentence)
        print("negative: {: <15} neutral: {: <15} positive: {: <15} compound: {}\n".format(
            scores['neg'], scores['neu'], scores['pos'], scores['compound']
        ))

In [ ]:
sent_by_sent(book_1_sent)

In [ ]:
sent_by_sent(book_2_sent)

In [ ]:
sent_by_sent(book_3_sent)

In [ ]:
sent_by_sent(book_4_sent)

In [ ]:
sent_by_sent(book_5_sent)

In [ ]:
sent_by_sent(book_6_sent)

In [ ]:
sent_by_sent(book_7_sent)

<a id="5.2"></a>

<h1 style="background-color:#1A472A;font-family:newtimeroman;font-size:200%;text-align:center;border-radius:15px 50px;"><span style="color:#DCDCDC">Sentiment Analysis For Each Chapter</span></h1>

In [ ]:
# fig, ax = plt.subplots(2,4)
# plt.rcParams.update({"figure.figsize":(10,7), "figure.dpi":100})

In [ ]:
# Store sentiment scores for each chapter in a list
def chapter_score(book_chapters):
    chapter_scores = []
    for chapter in book_chapters:
        sentences = sent_tokenize(chapter)

        # Calculate average score, we will first store scores for all sentences
        all_scores = []    
        for sentence in sentences:
            scores = sentiment_model.polarity_scores(sentence)
            compound_score = scores['compound']
            all_scores.append(compound_score)

        # Average is calculated as sum of all scores divided by number of scores
        chapter_score = sum(all_scores) / len(all_scores)
        chapter_scores.append(chapter_score)
    return chapter_scores

# Plot sentiment scores for each chapter by book
def plot_sent_scores(chapter_names, chapter_scores, i):
    plt.figure(figsize=(15, 8))    
    pos = list(range(len(chapter_scores)))   
    plt.plot(pos, list(chapter_scores))   
    plt.xticks(pos, chapter_names, rotation=90)
    plt.title(f"Sentiment Throughout Book {i} By Chapters", fontsize=18)
    plt.show()

In [ ]:
# Plot sentiment score for book 1
book_1_scores = chapter_score(book_1_chapters)
plot_sent_scores(book_1_chapter_names, book_1_scores,1)

# Plot sentiment score for book 2
book_2_scores = chapter_score(book_2_chapters)
plot_sent_scores(book_2_chapter_names, book_2_scores,2)

# Plot sentiment score for book 3
book_3_scores = chapter_score(book_3_chapters)
plot_sent_scores(book_3_chapter_names, book_3_scores,3)

# Plot sentiment score for book 4
book_4_scores = chapter_score(book_4_chapters)
plot_sent_scores(book_4_chapter_names, book_4_scores,4)

# Plot sentiment score for book 5
book_5_scores = chapter_score(book_5_chapters)
plot_sent_scores(book_5_chapter_names, book_5_scores,5)

# Plot sentiment score for book 6
book_6_scores = chapter_score(book_6_chapters)
plot_sent_scores(book_6_chapter_names, book_6_scores,6)

# Plot sentiment score for book 7
book_7_scores = chapter_score(book_7_chapters)
plot_sent_scores(book_7_chapter_names, book_7_scores,7)

<a id="5.3"></a>

<h1 style="background-color:#1A472A;font-family:newtimeroman;font-size:200%;text-align:center;border-radius:15px 50px;"><span style="color:#DCDCDC">Sentiment Analysis For Entire Series</span></h1>

In [ ]:
# Store sentiment scores for all chapters in a list
all_chapters, all_names = load_all_chapters()

all_chapter_scores = []

for book_chapters in all_chapters:
    chapter_scores = []
    for chapter in book_chapters:
        sentences = sent_tokenize(chapter)
        all_scores = []    
        for sentence in sentences:
            scores = sentiment_model.polarity_scores(sentence)
            compound_score = scores['compound']
            all_scores.append(compound_score)

        # Average is calculated as sum of all scores divided by number of scores
        chapter_score = sum(all_scores) / len(all_scores)
        chapter_scores.append(chapter_score)
        
    all_chapter_scores.append(chapter_scores)

In [ ]:
# Plot sentiment scores for all chapters all books
def plot_all_book_scores(all_names, all_scores):
    plt.figure(figsize=(20, 10))
    c = ['blue', 'green', 'yellow', 'red', 'black', 'orange', 'purple']
    
    previous = 0
    
    for idx, scores in enumerate(all_scores):
        pos = [i + previous for i in range(len(scores))]
        previous += len(scores)
        plt.plot(pos, scores, color=c[idx])

    plt.xlim((-0.5, previous + 0.5))
    xticks = [n for ns in all_names for n in ns]
    plt.xticks(list(range(len(xticks))), xticks, rotation=90, fontsize=6)
    plt.tight_layout()
    plt.savefig('all_chapters.pdf')
    plt.show()
    
plot_all_book_scores(all_names, all_chapter_scores)

In [ ]:
def sentiment_rating(chapter_sentences):
    polar=sentiment_model.polarity_scores(chapter_sentences)
    comp=polar['compound']
    return comp
    
df['Vader_sentiment']=df['sentence'].apply(sentiment_rating)

def sentiment_polarity(comp):
    if comp < 0:
        return  'negative'
    elif comp == 0:
        return  'neutral'
    else: 
        return 'positive'
df['Vader_polarity']=df['Vader_sentiment'].apply(sentiment_polarity)
def star_rating(compound):
    if compound <-0.25:
        return '1'
    elif compound > -0.25 and compound < 0:
        return '2'
    elif compound >=0 and compound < 0.25:
        return '3'
    elif compound > 0.25 and compound < 0.75:
        return '4'
    else: 
        return '5'
df['Vader_star']=df['Vader_sentiment'].apply(star_rating)
df

In [ ]:
def sentiment_analysis(text):

    # Create a function to get the subjectivity
    def getSubjectivity(text):
        return TextBlob(text).sentiment.subjectivity
  
    # Create a function to get the polarity
    def getPolarity(text):
        return TextBlob(text).sentiment.polarity
  
    # Create two new columns ‘Subjectivity’ & ‘Polarity’
    df['TextBlob_Subjectivity'] = text.apply(getSubjectivity)
    df['TextBlob_Polarity'] = text.apply(getPolarity)
 
    #Transform scores to text-based sentiment labels
    def getAnalysis(score):
        if score < 0:
            return 'negative'
        elif score == 0:
            return 'neutral'
        else:
            return 'positive'
    
    df['TextBlob_Analysis'] = df['TextBlob_Polarity'].apply(getAnalysis)
    
    return df

In [ ]:
sentiment_analysis(df['sentence'])

<a id="6"></a>

<h1 style="background-color:#740001;font-family:newtimeroman;font-size:225%;text-align:center;border-radius:15px 50px;"><span style="color:#EEBA30">LDA Topic Modeling</span></h1>

In [ ]:
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words=all_stpwrd)

In [ ]:
dtm = cv.fit_transform(book_1_chapters)

In [ ]:
LDA = LatentDirichletAllocation(n_components=len(book_1_chapters),random_state=42)
LDA.fit(dtm)

In [ ]:
for index,topic in enumerate(LDA.components_):
    print(f'TOP 20 WORDS FOR TOPIC #{index+1}')
    print([cv.get_feature_names_out()[i] for i in topic.argsort()[-20:]])
    print('\n')